<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Project-Objective" data-toc-modified-id="Project-Objective-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Project Objective</a></span></li><li><span><a href="#Hypothesis-&amp;-Questions" data-toc-modified-id="Hypothesis-&amp;-Questions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Hypothesis &amp; Questions</a></span><ul class="toc-item"><li><span><a href="#Hypotheses" data-toc-modified-id="Hypotheses-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Hypotheses</a></span></li><li><span><a href="#Questions" data-toc-modified-id="Questions-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Questions</a></span></li></ul></li><li><span><a href="#Answering-the-other-questions" data-toc-modified-id="Answering-the-other-questions-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Answering the other questions</a></span></li><li><span><a href="#Feature-Engineering" data-toc-modified-id="Feature-Engineering-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Feature Engineering</a></span></li><li><span><a href="#Modelling" data-toc-modified-id="Modelling-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Modelling</a></span><ul class="toc-item"><li><span><a href="#Model-2:-Random-Forest-Model" data-toc-modified-id="Model-2:-Random-Forest-Model-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Model 2: Random Forest Model</a></span></li></ul></li><li><span><a href="#Exporting" data-toc-modified-id="Exporting-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Exporting</a></span></li></ul></div>

## Project Objective

To build a model that accurately predicts the unit sales for the items sold by Corporation Favorita

## Hypothesis & Questions

### Hypotheses

### Questions

1. Is the train dataset complete (has all the required dates)?
2. Which dates have the lowest and highest sales for each year?
3. Did the earthquake impact sales?
4. Are certain groups of stores selling more products? (Cluster, city, state, type)
5. Are sales affected by promotions, oil prices and holidays?
6. What analysis can we get from the date and its extractable features?
7. What is the difference between RMSLE, RMSE, MSE (or why is the MAE greater than all of them?)

In [173]:
# Extracting the Zip File to Get Access to the Data
import zipfile
with zipfile.ZipFile("store-sales-time-series-forecasting.zip", "r") as zip_loaded:
    zip_loaded.extractall("files/")

print("Extraction Complete.")

Extraction Complete.


In [174]:
# Importing and loading relevant libraries and packages
import numpy as np
import pandas as pd
import re

import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline

from itertools import *
from sklearn import *

from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.tree import DecisionTreeRegressor

import xgboost as xgb
from xgboost import XGBRegressor

# Hiding warnings
import warnings
warnings.filterwarnings("ignore")

print("Loading complete.", "Warnings hidden.")

Loading complete. Warnings hidden.


**Previewing & exploring the files**

**Train data and complementary data**

In [175]:
train_data = pd.read_csv("files/train.csv")
train_data

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.00,0
1,1,2013-01-01,1,BABY CARE,0.00,0
2,2,2013-01-01,1,BEAUTY,0.00,0
3,3,2013-01-01,1,BEVERAGES,0.00,0
4,4,2013-01-01,1,BOOKS,0.00,0
...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.13,0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.55,1
3000885,3000885,2017-08-15,9,PRODUCE,"2,419.73",148
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.00,8


In [176]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 137.4+ MB


In [177]:
train_data.nunique()

id             3000888
date              1684
store_nbr           54
family              33
sales           379610
onpromotion        362
dtype: int64

In [178]:
# Setting all floats to display with 2 decimal places
pd.options.display.float_format = "{:,.2f}".format

In [179]:
# Getting the  actual dates
actual_days = train_data["date"].unique()
actual_days

array(['2013-01-01', '2013-01-02', '2013-01-03', ..., '2017-08-13',
       '2017-08-14', '2017-08-15'], dtype=object)

In [180]:
# Converting the date column to datetime format
train_data["sales_date"] = pd.to_datetime(train_data["date"]).dt.date
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 7 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  int64  
 6   sales_date   object 
dtypes: float64(1), int64(3), object(3)
memory usage: 160.3+ MB


In [181]:
# Checking if there are any missing dates
date_range = train_data.sales_date.min(), train_data.sales_date.max()
date_range

(datetime.date(2013, 1, 1), datetime.date(2017, 8, 15))

In [182]:
# Check completeness of dates
# Number of expected dates
expected_days = pd.date_range(
    start=train_data["sales_date"].min(), end=train_data["sales_date"].max())
expected_days

DatetimeIndex(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04',
               '2013-01-05', '2013-01-06', '2013-01-07', '2013-01-08',
               '2013-01-09', '2013-01-10',
               ...
               '2017-08-06', '2017-08-07', '2017-08-08', '2017-08-09',
               '2017-08-10', '2017-08-11', '2017-08-12', '2017-08-13',
               '2017-08-14', '2017-08-15'],
              dtype='datetime64[ns]', length=1688, freq='D')

We note a difference of 4 days between the actual dates (1,684) and expected dates (1,688) within the range. As such we have to find the missing dates and add them to ensure completeness of the dates.

This gives the answer to question 1 (Is the train dataset complete (has all the required dates)?) as a no.

In [183]:
# Get missing dates
missing_dates = set(expected_days.date) - \
    set(train_data["sales_date"].unique())
missing_dates

{datetime.date(2013, 12, 25),
 datetime.date(2014, 12, 25),
 datetime.date(2015, 12, 25),
 datetime.date(2016, 12, 25)}

In [184]:
# Getting the list of unique stores
unique_stores = train_data["store_nbr"].unique()
unique_stores

array([ 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,  2, 20, 21, 22, 23, 24,
       25, 26, 27, 28, 29,  3, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39,  4,
       40, 41, 42, 43, 44, 45, 46, 47, 48, 49,  5, 50, 51, 52, 53, 54,  6,
        7,  8,  9], dtype=int64)

In [185]:
# Getting the unique families
unique_families = train_data["family"].unique()
unique_families

array(['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS',
       'BREAD/BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS',
       'FROZEN FOODS', 'GROCERY I', 'GROCERY II', 'HARDWARE',
       'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES',
       'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE',
       'LIQUOR,WINE,BEER', 'MAGAZINES', 'MEATS', 'PERSONAL CARE',
       'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'POULTRY',
       'PREPARED FOODS', 'PRODUCE', 'SCHOOL AND OFFICE SUPPLIES',
       'SEAFOOD'], dtype=object)

Since we're predicting the sales for each store, it means we have to fill in the missing dates for each store. We will do this with the _product_ module from _itertools_

In [186]:
missing_data = list(product(missing_dates, unique_stores, unique_families))
train_addon = pd.DataFrame(missing_data, columns=[
                           "sales_date", "store_nbr", "family"])
train_addon

,sales_date,store_nbr,family
0,2014-12-25,1,AUTOMOTIVE
1,2014-12-25,1,BABY CARE
2,2014-12-25,1,BEAUTY
3,2014-12-25,1,BEVERAGES
4,2014-12-25,1,BOOKS
...,...,...,...
7123,2016-12-25,9,POULTRY
7124,2016-12-25,9,PREPARED FOODS
7125,2016-12-25,9,PRODUCE
7126,2016-12-25,9,SCHOOL AND OFFICE SUPPLIES


In [187]:
train_data = pd.concat([train_data, train_addon], ignore_index=True)
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3008016 entries, 0 to 3008015
Data columns (total 7 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           float64
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  float64
 6   sales_date   object 
dtypes: float64(3), int64(1), object(3)
memory usage: 160.6+ MB


In [188]:
train_data

,id,date,store_nbr,family,sales,onpromotion,sales_date
0,0.00,2013-01-01,1,AUTOMOTIVE,0.00,0.00,2013-01-01
1,1.00,2013-01-01,1,BABY CARE,0.00,0.00,2013-01-01
2,2.00,2013-01-01,1,BEAUTY,0.00,0.00,2013-01-01
3,3.00,2013-01-01,1,BEVERAGES,0.00,0.00,2013-01-01
4,4.00,2013-01-01,1,BOOKS,0.00,0.00,2013-01-01
...,...,...,...,...,...,...,...
3008011,NaN,NaN,9,POULTRY,NaN,NaN,2016-12-25
3008012,NaN,NaN,9,PREPARED FOODS,NaN,NaN,2016-12-25
3008013,NaN,NaN,9,PRODUCE,NaN,NaN,2016-12-25
3008014,NaN,NaN,9,SCHOOL AND OFFICE SUPPLIES,NaN,NaN,2016-12-25


- With December 25 omitted from each of the years, I assume that it was deliberate - most likely because all shops are closed on December 25 each year. In effect, no items would have been on promotion and no sales would have been made; that is to say that it is safe to fill the null "sales" and "onpromotion" column data with 0.

- By this, I am also dropping the "id" column as it will not be relevant to subsequent analyses and modelling.

- I will be filling the missing dates in the original dates column with the sales data, for aesthetic purposes only.

In [189]:
# Dropping "id" and "date" columns
train_data.drop(columns=["id", "date"], axis=1, inplace=True)

train_data["family"] = train_data["family"].str.title()

# Filling missing rows in the sales column and casting it to numeric
train_data["sales"].fillna(0, inplace=True)
train_data["sales"] = pd.to_numeric(train_data["sales"])

# Filling missing rows in the onpromotion column
train_data["onpromotion"].fillna(0, inplace=True)

train_data

,store_nbr,family,sales,onpromotion,sales_date
0,1,Automotive,0.00,0.00,2013-01-01
1,1,Baby Care,0.00,0.00,2013-01-01
2,1,Beauty,0.00,0.00,2013-01-01
3,1,Beverages,0.00,0.00,2013-01-01
4,1,Books,0.00,0.00,2013-01-01
...,...,...,...,...,...
3008011,9,Poultry,0.00,0.00,2016-12-25
3008012,9,Prepared Foods,0.00,0.00,2016-12-25
3008013,9,Produce,0.00,0.00,2016-12-25
3008014,9,School And Office Supplies,0.00,0.00,2016-12-25


**Transactions data**

In [190]:
transactions = pd.read_csv("files/transactions.csv")
transactions

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922
...,...,...,...
83483,2017-08-15,50,2804
83484,2017-08-15,51,1573
83485,2017-08-15,52,2255
83486,2017-08-15,53,932


In [191]:
# Viewing basic information about the transactions data
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          83488 non-null  object
 1   store_nbr     83488 non-null  int64 
 2   transactions  83488 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.9+ MB


In [192]:
transactions.nunique()

date            1682
store_nbr         54
transactions    4993
dtype: int64

- Since the train data has the same number of unique stores as the transactions data, we can use the unique stores variable defined earlier to fill in the missing dates.
- Also, given that the transactions and train data cover the same period, it is concerning that the transactions data has even less unique dates than the train data has. As such, we have to find and impute the missing dates as done for the train data.

In [193]:
# Getting missing dates
transactions["sales_date"] = pd.to_datetime(transactions["date"]).dt.date
missing_txn_dates = set(expected_days.date) - \
    set(transactions["sales_date"].unique())
missing_txn_dates

{datetime.date(2013, 12, 25),
 datetime.date(2014, 12, 25),
 datetime.date(2015, 12, 25),
 datetime.date(2016, 1, 1),
 datetime.date(2016, 1, 3),
 datetime.date(2016, 12, 25)}

In [194]:
missing_txn_data = list(product(missing_txn_dates, unique_stores))
txn_data_addon = pd.DataFrame(missing_txn_data, columns=[
                              "sales_date", "store_nbr"])
txn_data_addon

,sales_date,store_nbr
0,2015-12-25,1
1,2015-12-25,10
2,2015-12-25,11
3,2015-12-25,12
4,2015-12-25,13
...,...,...
319,2013-12-25,54
320,2013-12-25,6
321,2013-12-25,7
322,2013-12-25,8


In [195]:
transactions

,date,store_nbr,transactions,sales_date
0,2013-01-01,25,770,2013-01-01
1,2013-01-02,1,2111,2013-01-02
2,2013-01-02,2,2358,2013-01-02
3,2013-01-02,3,3487,2013-01-02
4,2013-01-02,4,1922,2013-01-02
...,...,...,...,...
83483,2017-08-15,50,2804,2017-08-15
83484,2017-08-15,51,1573,2017-08-15
83485,2017-08-15,52,2255,2017-08-15
83486,2017-08-15,53,932,2017-08-15


In [196]:
# Adding the data for the missing transaction dates to the main transaction data and filling nulls with 0
transactions = pd.concat([transactions, txn_data_addon], ignore_index=True)
transactions.drop("date", axis=1, inplace=True)
transactions["transactions"].fillna(0, inplace=True)

In [197]:
# Recasting the sales date column data type to date
transactions["sales_date"] = pd.to_datetime(transactions["sales_date"]).dt.date
transactions

,store_nbr,transactions,sales_date
0,25,770.00,2013-01-01
1,1,"2,111.00",2013-01-02
2,2,"2,358.00",2013-01-02
3,3,"3,487.00",2013-01-02
4,4,"1,922.00",2013-01-02
...,...,...,...
83807,54,0.00,2013-12-25
83808,6,0.00,2013-12-25
83809,7,0.00,2013-12-25
83810,8,0.00,2013-12-25


In [198]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83812 entries, 0 to 83811
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   store_nbr     83812 non-null  int64  
 1   transactions  83812 non-null  float64
 2   sales_date    83812 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 1.9+ MB


**Holidays and events data**

In [199]:
holidays_events = pd.read_csv("files/holidays_events.csv")
holidays_events

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
...,...,...,...,...,...,...
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False
348,2017-12-25,Holiday,National,Ecuador,Navidad,False


In [200]:
holidays_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         350 non-null    object
 1   type         350 non-null    object
 2   locale       350 non-null    object
 3   locale_name  350 non-null    object
 4   description  350 non-null    object
 5   transferred  350 non-null    bool  
dtypes: bool(1), object(5)
memory usage: 14.1+ KB


The holidays and events dataframe looks complete, hence there will be no need for any cleaning now.

In [201]:
holidays_events["date"] = pd.to_datetime(holidays_events["date"]).dt.date
holidays_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         350 non-null    object
 1   type         350 non-null    object
 2   locale       350 non-null    object
 3   locale_name  350 non-null    object
 4   description  350 non-null    object
 5   transferred  350 non-null    bool  
dtypes: bool(1), object(5)
memory usage: 14.1+ KB


In [202]:
holidays_events.nunique()

date           312
type             6
locale           3
locale_name     24
description    103
transferred      2
dtype: int64

In [203]:
holidays_events["type"].unique()

array(['Holiday', 'Transfer', 'Additional', 'Bridge', 'Work Day', 'Event'],
      dtype=object)

In [204]:
holidays_events[holidays_events["type"] == "Work Day"]

,date,type,locale,locale_name,description,transferred
42,2013-01-05,Work Day,National,Ecuador,Recupero puente Navidad,False
43,2013-01-12,Work Day,National,Ecuador,Recupero puente primer dia del ano,False
149,2014-12-20,Work Day,National,Ecuador,Recupero Puente Navidad,False
161,2015-01-10,Work Day,National,Ecuador,Recupero Puente Primer dia del ano,False
283,2016-11-12,Work Day,National,Ecuador,Recupero Puente Dia de Difuntos,False


In [205]:
# Getting missing dates
missing_holiday_dates = set(expected_days.date) - \
    set(holidays_events["date"].unique())
missing_holiday_dates

{datetime.date(2013, 4, 6),
 datetime.date(2014, 8, 30),
 datetime.date(2014, 3, 25),
 datetime.date(2013, 3, 6),
 datetime.date(2016, 5, 20),
 datetime.date(2014, 12, 2),
 datetime.date(2014, 2, 13),
 datetime.date(2017, 2, 11),
 datetime.date(2014, 11, 23),
 datetime.date(2016, 2, 14),
 datetime.date(2013, 11, 30),
 datetime.date(2016, 7, 8),
 datetime.date(2014, 5, 27),
 datetime.date(2016, 3, 3),
 datetime.date(2017, 4, 9),
 datetime.date(2013, 4, 30),
 datetime.date(2017, 2, 16),
 datetime.date(2013, 1, 25),
 datetime.date(2013, 6, 19),
 datetime.date(2013, 3, 13),
 datetime.date(2017, 1, 31),
 datetime.date(2013, 3, 31),
 datetime.date(2017, 4, 2),
 datetime.date(2015, 8, 25),
 datetime.date(2016, 3, 28),
 datetime.date(2015, 5, 2),
 datetime.date(2016, 6, 29),
 datetime.date(2013, 6, 12),
 datetime.date(2016, 2, 4),
 datetime.date(2014, 7, 21),
 datetime.date(2016, 12, 27),
 datetime.date(2017, 1, 28),
 datetime.date(2013, 3, 1),
 datetime.date(2015, 11, 28),
 datetime.date(2016

In [206]:
# Creating a dataframe for the missing dates in the holiday data
holidays_add = pd.DataFrame(missing_holiday_dates, columns=["date"])
holidays_add

,date
0,2013-04-06
1,2014-08-30
2,2014-03-25
3,2013-03-06
4,2016-05-20
...,...
1427,2014-04-27
1428,2015-11-01
1429,2013-02-06
1430,2013-01-17


In [207]:
# Adding the  missing holiday dates to the main dataframe
holidays_events = pd.concat([holidays_events, holidays_add], ignore_index=True)
holidays_events["date"] = pd.to_datetime(holidays_events["date"]).dt.date
holidays_events = holidays_events.sort_values(by=["date"], ignore_index=True)
holidays_events

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
...,...,...,...,...,...,...
1777,2017-12-22,Holiday,Local,Salinas,Cantonizacion de Salinas,False
1778,2017-12-23,Additional,National,Ecuador,Navidad-2,False
1779,2017-12-24,Additional,National,Ecuador,Navidad-1,False
1780,2017-12-25,Holiday,National,Ecuador,Navidad,False


**Oil data**

In [208]:
oil_data = pd.read_csv("files/oil.csv")
oil_data.head(10)

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
5,2013-01-08,93.21
6,2013-01-09,93.08
7,2013-01-10,93.81
8,2013-01-11,93.60
9,2013-01-14,94.27


In [209]:
oil_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1218 non-null   object 
 1   dcoilwtico  1175 non-null   float64
dtypes: float64(1), object(1)
memory usage: 19.2+ KB


We note about 43 missing values for oil prices in the oil data. Checks online revealed that said data were unavailable in real time, as such a forward fill method will be applied to fill the nulls and a backfill applied to fill any rows missing after that.

In [210]:
# Filling nulls with interpolation, then any remainders with forward fill and backfill
oil_data = oil_data.interpolate()
oil_data = oil_data.ffill().bfill()
oil_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1218 non-null   object 
 1   dcoilwtico  1218 non-null   float64
dtypes: float64(1), object(1)
memory usage: 19.2+ KB


In [211]:
oil_data.head(10)

,date,dcoilwtico
0,2013-01-01,93.14
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
5,2013-01-08,93.21
6,2013-01-09,93.08
7,2013-01-10,93.81
8,2013-01-11,93.60
9,2013-01-14,94.27


The oil data now has no nulls, and is supposed to be complete, but we note that there are still some missing dates. e.g. it moves from January 4, 2013 to January 7, 2013. A quick check reveals that those dates are weekends, implying that the data is for business days and does not include weekends. With this in mind, I assume that oil prices, for the period, are frozen at close of business days of Friday and so remain constant over the weekends. As such, the "missing dates" (weekends) can be brought in another forward fills applied to them.

In [212]:
# Converting the dates in the oil data to dates
oil_data["date"] = pd.to_datetime(oil_data["date"]).dt.date

# Getting missing dates
missing_oil_dates = set(expected_days.date) - set(oil_data["date"].unique())
missing_oil_dates

{datetime.date(2013, 1, 5),
 datetime.date(2013, 1, 6),
 datetime.date(2013, 1, 12),
 datetime.date(2013, 1, 13),
 datetime.date(2013, 1, 19),
 datetime.date(2013, 1, 20),
 datetime.date(2013, 1, 26),
 datetime.date(2013, 1, 27),
 datetime.date(2013, 2, 2),
 datetime.date(2013, 2, 3),
 datetime.date(2013, 2, 9),
 datetime.date(2013, 2, 10),
 datetime.date(2013, 2, 16),
 datetime.date(2013, 2, 17),
 datetime.date(2013, 2, 23),
 datetime.date(2013, 2, 24),
 datetime.date(2013, 3, 2),
 datetime.date(2013, 3, 3),
 datetime.date(2013, 3, 9),
 datetime.date(2013, 3, 10),
 datetime.date(2013, 3, 16),
 datetime.date(2013, 3, 17),
 datetime.date(2013, 3, 23),
 datetime.date(2013, 3, 24),
 datetime.date(2013, 3, 30),
 datetime.date(2013, 3, 31),
 datetime.date(2013, 4, 6),
 datetime.date(2013, 4, 7),
 datetime.date(2013, 4, 13),
 datetime.date(2013, 4, 14),
 datetime.date(2013, 4, 20),
 datetime.date(2013, 4, 21),
 datetime.date(2013, 4, 27),
 datetime.date(2013, 4, 28),
 datetime.date(2013, 5, 

In [213]:
oil_dates_add = pd.DataFrame(missing_oil_dates, columns=["date"])
oil_dates_add

,date
0,2017-03-05
1,2013-04-06
2,2014-08-30
3,2016-06-26
4,2014-03-22
...,...
477,2014-11-29
478,2014-04-27
479,2015-11-01
480,2014-07-06


In [214]:
# Adding the  missing oil dates to the main dataframe
oil_data = pd.concat([oil_data, oil_dates_add], ignore_index=True)
oil_data["date"] = pd.to_datetime(oil_data["date"])
oil_data = oil_data.sort_values(by=["date"], ignore_index=True)
oil_data.head(10)

,date,dcoilwtico
0,2013-01-01,93.14
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-05,NaN
5,2013-01-06,NaN
6,2013-01-07,93.20
7,2013-01-08,93.21
8,2013-01-09,93.08
9,2013-01-10,93.81


In [215]:
# Filling nulls with forward fill and backfill
oil_data = oil_data.ffill().bfill()
oil_data.head(10)

,date,dcoilwtico
0,2013-01-01,93.14
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-05,93.12
5,2013-01-06,93.12
6,2013-01-07,93.20
7,2013-01-08,93.21
8,2013-01-09,93.08
9,2013-01-10,93.81


In [216]:
# Recasting the oil data dates to datetime dates
oil_data["date"] = pd.to_datetime(oil_data["date"]).dt.date
oil_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1700 entries, 0 to 1699
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1700 non-null   object 
 1   dcoilwtico  1700 non-null   float64
dtypes: float64(1), object(1)
memory usage: 26.7+ KB


**Stores data**

In [217]:
stores_data = pd.read_csv("files/stores.csv")
stores_data.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [218]:
stores_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   store_nbr  54 non-null     int64 
 1   city       54 non-null     object
 2   state      54 non-null     object
 3   type       54 non-null     object
 4   cluster    54 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 2.2+ KB


**Test data**

In [219]:
test_data = pd.read_csv("files/test.csv")
test_data

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0
...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1
28508,3029396,2017-08-31,9,PREPARED FOODS,0
28509,3029397,2017-08-31,9,PRODUCE,1
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9


In [220]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28512 entries, 0 to 28511
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           28512 non-null  int64 
 1   date         28512 non-null  object
 2   store_nbr    28512 non-null  int64 
 3   family       28512 non-null  object
 4   onpromotion  28512 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 1.1+ MB


The test data looks complete, with no nulls. Casting the date column to date will be the only cleaning activity here.

In [221]:
# Casting the date column to date data type
test_data["date"] = pd.to_datetime(test_data["date"]).dt.date

**Sample Submission**

In [222]:
sample_submission = pd.read_csv("files/sample_submission.csv")
sample_submission

,id,sales
0,3000888,0.00
1,3000889,0.00
2,3000890,0.00
3,3000891,0.00
4,3000892,0.00
...,...,...
28507,3029395,0.00
28508,3029396,0.00
28509,3029397,0.00
28510,3029398,0.00


No changes will be made to the sample submission as it is only a guide.

## Answering the other questions

**Which dates have the lowest and highest sales for each year?**

The imputation of the originally missing dates means that automatically, minimum sales for each of the four years will be on those dates (December 25 each year), but that is not what we want. What we want to know is which days had the least sales when stores were opened, as such I will only include sales values greater than 0.

In [223]:
# Aggregating sales by dates
train_by_date = train_data[train_data["sales"] != 0.00
                           ].groupby(by="sales_date").sales.agg(["sum"]
                                                                ).sort_values(by="sales_date")
train_by_date

,sum
sales_date,
2013-01-01,"2,511.62"
2013-01-02,"496,092.42"
2013-01-03,"361,461.23"
2013-01-04,"354,459.68"
2013-01-05,"477,350.12"
...,...
2017-08-11,"826,373.72"
2017-08-12,"792,630.54"
2017-08-13,"865,639.68"


In [224]:
# Creating a column for the years for grouping
train_by_date["year"] = pd.to_datetime(train_by_date.index).year
train_by_date.rename(columns={"sum": "total_sales"}, inplace=True)
train_by_date = train_by_date.reset_index()
train_by_date

,sales_date,total_sales,year
0,2013-01-01,"2,511.62",2013
1,2013-01-02,"496,092.42",2013
2,2013-01-03,"361,461.23",2013
3,2013-01-04,"354,459.68",2013
4,2013-01-05,"477,350.12",2013
...,...,...,...
1679,2017-08-11,"826,373.72",2017
1680,2017-08-12,"792,630.54",2017
1681,2017-08-13,"865,639.68",2017
1682,2017-08-14,"760,922.41",2017


In [225]:
fig = px.line(train_by_date, x="sales_date", y="total_sales",
              title="Sales trend for Corporation Favorita from 2013 - 2017",
              labels={"sales_date": "Sales Date", "total_sales": "Total Sales"})
fig.show()

In [226]:
data_2013 = train_by_date[train_by_date["year"] == 2013]
data_2013 = data_2013.reset_index()
data_2013

,index,sales_date,total_sales,year
0,0,2013-01-01,"2,511.62",2013
1,1,2013-01-02,"496,092.42",2013
2,2,2013-01-03,"361,461.23",2013
3,3,2013-01-04,"354,459.68",2013
4,4,2013-01-05,"477,350.12",2013
...,...,...,...,...
359,359,2013-12-27,"479,314.97",2013
360,360,2013-12-28,"556,952.31",2013
361,361,2013-12-29,"499,719.50",2013
362,362,2013-12-30,"635,134.74",2013


In [227]:
fig = px.line(data_2013, x="sales_date", y="total_sales", title="Sales trend for Corporation Favorita in 2013",
              labels={"sales_date": "Sales Date", "total_sales": "Total Sales"})
fig.show()

In [228]:
min_sales_13 = data_2013["total_sales"].min()
max_sales_13 = data_2013["total_sales"].max()
low_hi_sales_13 = data_2013[(data_2013["total_sales"] == min_sales_13) | (
    data_2013["total_sales"] == max_sales_13)]
low_hi_sales_13

,index,sales_date,total_sales,year
0,0,2013-01-01,"2,511.62",2013
356,356,2013-12-23,"792,865.28",2013


In [229]:
data_2014 = train_by_date[train_by_date["year"] == 2014]
data_2014 = data_2014.reset_index()
data_2014

,index,sales_date,total_sales,year
0,364,2014-01-01,"8,602.07",2014
1,365,2014-01-02,"801,011.23",2014
2,366,2014-01-03,"680,672.85",2014
3,367,2014-01-04,"936,628.89",2014
4,368,2014-01-05,"949,618.79",2014
...,...,...,...,...
359,723,2014-12-27,"740,596.16",2014
360,724,2014-12-28,"716,329.64",2014
361,725,2014-12-29,"773,998.40",2014
362,726,2014-12-30,"912,970.53",2014


In [230]:
fig = px.line(data_2014, x="sales_date", y="total_sales", title="Sales trend for Corporation Favorita in 2014",
              labels={"sales_date": "Sales Date", "total_sales": "Total Sales"})
fig.show()

In [231]:
min_sales_14 = data_2014["total_sales"].min()
max_sales_14 = data_2014["total_sales"].max()
low_hi_sales_14 = data_2014[(data_2014["total_sales"] == min_sales_14) | (
    data_2014["total_sales"] == max_sales_14)]
low_hi_sales_14

,index,sales_date,total_sales,year
0,364,2014-01-01,"8,602.07",2014
356,720,2014-12-23,"1,064,977.97",2014


In [232]:
data_2015 = train_by_date[train_by_date["year"] == 2015]
data_2015 = data_2015.reset_index()
data_2015

,index,sales_date,total_sales,year
0,728,2015-01-01,"12,773.62",2015
1,729,2015-01-02,"657,763.39",2015
2,730,2015-01-03,"648,880.69",2015
3,731,2015-01-04,"730,923.78",2015
4,732,2015-01-05,"569,267.30",2015
...,...,...,...,...
359,1087,2015-12-27,"837,714.13",2015
360,1088,2015-12-28,"789,684.91",2015
361,1089,2015-12-29,"870,762.03",2015
362,1090,2015-12-30,"1,030,043.74",2015


In [233]:
fig = px.line(data_2015, x="sales_date", y="total_sales", title="Sales trend for Corporation Favorita in 2015",
              labels={"sales_date": "Sales Date", "total_sales": "Total Sales"})
fig.show()

In [234]:
min_sales_15 = data_2015["total_sales"].min()
max_sales_15 = data_2015["total_sales"].max()
low_hi_sales_15 = data_2015[(data_2015["total_sales"] == min_sales_15) | (
    data_2015["total_sales"] == max_sales_15)]
low_hi_sales_15

,index,sales_date,total_sales,year
0,728,2015-01-01,"12,773.62",2015
276,1004,2015-10-04,"1,234,130.94",2015


In [235]:
data_2016 = train_by_date[train_by_date["year"] == 2016]
data_2016 = data_2016.reset_index()
data_2016

,index,sales_date,total_sales,year
0,1092,2016-01-01,"16,433.39",2016
1,1093,2016-01-02,"1,066,677.42",2016
2,1094,2016-01-03,"1,226,735.72",2016
3,1095,2016-01-04,"955,956.88",2016
4,1096,2016-01-05,"835,320.44",2016
...,...,...,...,...
360,1452,2016-12-27,"842,475.49",2016
361,1453,2016-12-28,"951,533.71",2016
362,1454,2016-12-29,"894,108.24",2016
363,1455,2016-12-30,"1,163,643.04",2016


In [236]:
fig = px.line(data_2016, x="sales_date", y="total_sales", title="Sales trend for Corporation Favorita in 2016",
              labels={"sales_date": "Sales Date", "total_sales": "Total Sales"})
fig.show()

In [237]:
min_sales_16 = data_2016["total_sales"].min()
max_sales_16 = data_2016["total_sales"].max()
low_hi_sales_16 = data_2016[(data_2016["total_sales"] == min_sales_16) | (
    data_2016["total_sales"] == max_sales_16)]
low_hi_sales_16

,index,sales_date,total_sales,year
0,1092,2016-01-01,"16,433.39",2016
108,1200,2016-04-18,"1,345,920.60",2016


In [238]:
data_2017 = train_by_date[train_by_date["year"] == 2017]
data_2017 = data_2017.reset_index()
data_2017

,index,sales_date,total_sales,year
0,1457,2017-01-01,"12,082.50",2017
1,1458,2017-01-02,"1,402,306.37",2017
2,1459,2017-01-03,"1,104,377.08",2017
3,1460,2017-01-04,"990,093.46",2017
4,1461,2017-01-05,"777,620.95",2017
...,...,...,...,...
222,1679,2017-08-11,"826,373.72",2017
223,1680,2017-08-12,"792,630.54",2017
224,1681,2017-08-13,"865,639.68",2017
225,1682,2017-08-14,"760,922.41",2017


In [239]:
fig = px.line(data_2017, x="sales_date", y="total_sales", title="Sales trend for Corporation Favorita in 2017",
              labels={"sales_date": "Sales Date", "total_sales": "Total Sales"})
fig.show()

In [240]:
min_sales_17 = data_2017["total_sales"].min()
max_sales_17 = data_2017["total_sales"].max()
low_hi_sales_17 = data_2017[(data_2017["total_sales"] == min_sales_17) | (
    data_2017["total_sales"] == max_sales_17)]
low_hi_sales_17

,index,sales_date,total_sales,year
0,1457,2017-01-01,"12,082.50",2017
90,1547,2017-04-01,"1,463,083.96",2017


In [241]:
# Combining the highest and lowest sales dates
low_hi_sales_df = pd.concat([low_hi_sales_13, low_hi_sales_14, low_hi_sales_15,
                             low_hi_sales_16, low_hi_sales_17], ignore_index=True)
low_hi_sales_df

,index,sales_date,total_sales,year
0,0,2013-01-01,"2,511.62",2013
1,356,2013-12-23,"792,865.28",2013
2,364,2014-01-01,"8,602.07",2014
3,720,2014-12-23,"1,064,977.97",2014
4,728,2015-01-01,"12,773.62",2015
5,1004,2015-10-04,"1,234,130.94",2015
6,1092,2016-01-01,"16,433.39",2016
7,1200,2016-04-18,"1,345,920.60",2016
8,1457,2017-01-01,"12,082.50",2017
9,1547,2017-04-01,"1,463,083.96",2017


The table above summarizes the dates which had least and most sales for each year. We note that Corporation Favorita made least sales on January 1 each year. For 2013 and 2014, they made most sales in December, while they made most sales in April 2016 and 2017. The outsider is 2015, when they made most sales in October.

**Did the earthquake impact sales?**

Given the foregoing - that 2016 sales peaked in April (18th April) - it is safe to assume that the magnitude 7.8 earthquake that struck Ecuador on April 16, 2016 affected sales positively.

The sales for 2016 will be visualized to give a closer look at the effects of the earthquake on the sales of Corporation Favorita.

In [242]:
# Visualizing the sales trend in 2016
fig = px.line(data_2016, x="sales_date", y="total_sales", title="Sales trend for Corporation Favorita in 2016",
              labels={"sales_date": "Sales Date", "total_sales": "Total Sales"})
fig.show()

From the graph above, we note that but for the earthquake sales would have peaked in December as it had in 2014 and 2013. But this time, it peaked in April. Sales hit USD 1.2m on April 17, the day after the earthquake and rose further to it's peak - USD 1.35m before normalizing.

For context, it is worthy of note that in all the previous years, sales would usually dip in April and May before picking up in June and the second half of the year.

Therefore, we can conclude that the earthquake impacted sales in 2016.

**Are certain groups of stores selling more products? (Cluster, city, state, type)**

To perform these analyses, the respective dataframes containing the data will have to be joined to the main train dataframe. This will be done in parts to prevent any memory errors.

***Analysis of sales by store cluster***

In [243]:
train_data.head(10)

,store_nbr,family,sales,onpromotion,sales_date
0,1,Automotive,0.00,0.00,2013-01-01
1,1,Baby Care,0.00,0.00,2013-01-01
2,1,Beauty,0.00,0.00,2013-01-01
3,1,Beverages,0.00,0.00,2013-01-01
4,1,Books,0.00,0.00,2013-01-01
5,1,Bread/Bakery,0.00,0.00,2013-01-01
6,1,Celebration,0.00,0.00,2013-01-01
7,1,Cleaning,0.00,0.00,2013-01-01
8,1,Dairy,0.00,0.00,2013-01-01
9,1,Deli,0.00,0.00,2013-01-01


In [244]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3008016 entries, 0 to 3008015
Data columns (total 5 columns):
 #   Column       Dtype  
---  ------       -----  
 0   store_nbr    int64  
 1   family       object 
 2   sales        float64
 3   onpromotion  float64
 4   sales_date   object 
dtypes: float64(2), int64(1), object(2)
memory usage: 114.7+ MB


In [245]:
stores_data.head(10)

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4
5,6,Quito,Pichincha,D,13
6,7,Quito,Pichincha,D,8
7,8,Quito,Pichincha,D,8
8,9,Quito,Pichincha,B,6
9,10,Quito,Pichincha,C,15


In [246]:
# How many unique values are there in the stores data
stores_data.nunique()

store_nbr    54
city         22
state        16
type          5
cluster      17
dtype: int64

In [247]:
# How many stores are there per cluster
cluster_check = stores_data.groupby(by="cluster").store_nbr.agg(["count"])
cluster_check = cluster_check.reset_index()
cluster_check["cluster"] = (cluster_check["cluster"]).apply(str)
cluster_check

,cluster,count
0,1,3
1,2,2
2,3,7
3,4,3
4,5,1
5,6,6
6,7,2
7,8,3
8,9,2
9,10,6


Key notes:
- Corporation Favorita has stores in 22 different cities across 16 states.
- The stores are grouped into 5 types and 17 clusters.
- Cluster 3 has the most stores (7), followed by clusters 6 and 10 with 6 stores each. The top 5 is completed by Cluster 15 with 5 stores, and clusters 13 and 14 with 4 stores each.

Does the number of stores in a cluster affect sales? Let's find out!

*Since the shared column in both dataframes is the store_nbr, we join on that column and aggregate by cluster.*

In [248]:
# Joining the train data with the stores data
train_by_cluster = pd.merge(train_data, stores_data, on="store_nbr")
train_by_cluster["year"] = pd.to_datetime(train_by_cluster["sales_date"]).dt.year
train_by_cluster["year"] = (train_by_cluster["year"]).apply(str)
train_by_cluster

,store_nbr,family,sales,onpromotion,sales_date,city,state,type,cluster,year
0,1,Automotive,0.00,0.00,2013-01-01,Quito,Pichincha,D,13,2013
1,1,Baby Care,0.00,0.00,2013-01-01,Quito,Pichincha,D,13,2013
2,1,Beauty,0.00,0.00,2013-01-01,Quito,Pichincha,D,13,2013
3,1,Beverages,0.00,0.00,2013-01-01,Quito,Pichincha,D,13,2013
4,1,Books,0.00,0.00,2013-01-01,Quito,Pichincha,D,13,2013
...,...,...,...,...,...,...,...,...,...,...
3008011,9,Poultry,0.00,0.00,2016-12-25,Quito,Pichincha,B,6,2016
3008012,9,Prepared Foods,0.00,0.00,2016-12-25,Quito,Pichincha,B,6,2016
3008013,9,Produce,0.00,0.00,2016-12-25,Quito,Pichincha,B,6,2016
3008014,9,School And Office Supplies,0.00,0.00,2016-12-25,Quito,Pichincha,B,6,2016


In [249]:
# Aggregating the combined stores and training data by cluster
sales_by_cluster = train_by_cluster.groupby(by=["cluster"]).sales.agg(["sum"])
sales_by_cluster = sales_by_cluster.reset_index().rename(
    columns={"sum": "total_sales"})
sales_by_cluster["cluster"] = (sales_by_cluster["cluster"]).apply(str)

# Joining with the count of stores per cluster
sales_by_cluster = sales_by_cluster.merge(
    cluster_check).rename(columns={"count": "store_count"})
sales_by_cluster

,cluster,total_sales,store_count
0,1,"54,376,751.96",3
1,2,"28,916,403.50",2
2,3,"75,628,703.46",7
3,4,"49,443,442.97",3
4,5,"62,087,553.25",1
5,6,"114,254,386.74",6
6,7,"15,478,120.49",2
7,8,"107,928,246.70",3
8,9,"30,561,081.05",2
9,10,"85,324,430.43",6


In [250]:
# Visualizing the total sales performance over the period by cluster
fig = px.bar(sales_by_cluster, x="cluster", y="total_sales",
             title="Sales Performance of Corporation Favorita's Store Clusters",
             labels={"cluster": "Cluster", "total_sales": "Total Sales"})
fig.show()

For total sales performance over the period, it is evident from the graph above that, cluster 14 was the best cluster with USD 157.43m. Cluster 6 came second with USD 114.25m, followed by Cluster 8 (USD 107.93m) and Cluster 11 (USD 100.61m) to complete the top 4. These stores crossed the USD 100m sales mark. Cluster 10 (USD 85.32) followed in the distance.

***Was total sales per cluster influenced by the number of stores in the cluster?***

In [251]:
# Visualizing the total sales performance over the period by cluster, coloured by number of stores per cluster
sales_by_cluster["store_count"] = sales_by_cluster["store_count"].apply(str)

fig = px.bar(sales_by_cluster, x="cluster", y="total_sales", color="store_count",
             category_orders={"cluster": ["1", "2", "3", "4", "5", "6",
                                          "7", "8", "9", "10", "11", "12",
                                          "13", "14", "15", "16", "17"],
                              "store_count": ["1", "2", "3", "4", "5", "6", "7"]},
             title="Sales Performance of Corporation Favorita's Store Clusters",
             labels={"cluster": "Cluster", "total_sales": "Total Sales", "store_count": "Stores in Cluster"})
fig.show()

From the graph above, we note that despite having the most number of stores, Cluster 3 was not in the top 5 clusters with most total sales. This is supported by Cluster 15's (5 stores) poor ranking in terms of total sales performance. 

Given that clusters 6 and 10, each with 6 stores, were both part of the top 5 clusters with most total sales, it becomes unclear if total sales per cluster was influenced by the numberof stores in the cluster.

In [252]:
# Visualizing the relationship between total sales performance and the number of stores per cluster
fig = px.scatter(sales_by_cluster, x="store_count", y="total_sales", trendline="ols",
                 category_orders={"cluster": ["1", "2", "3", "4", "5", "6", "7", "8", "9",
                                              "10", "11", "12", "13", "14", "15", "16", "17"],
                                  "store_count": ["1", "2", "3", "4", "5", "6", "7"]},
                 title="Sales Performance of Corporation Favorita's Store Clusters",
                 labels={"cluster": "Cluster", "total_sales": "Total Sales", "store_count": "Stores in Cluster"})
fig.show()

The plot above shows a weak positive relationship between the number of stores in a cluster and the total sales performance. This is confirmed by the R2 value of the trendline: 0.34636.

***Analysis of sales by city***

*How does the sales performance by city look like?*

In [253]:
# Joining the train data with the stores data
train_x_stores = pd.merge(train_data, stores_data, on="store_nbr")
train_x_stores

,store_nbr,family,sales,onpromotion,sales_date,city,state,type,cluster
0,1,Automotive,0.00,0.00,2013-01-01,Quito,Pichincha,D,13
1,1,Baby Care,0.00,0.00,2013-01-01,Quito,Pichincha,D,13
2,1,Beauty,0.00,0.00,2013-01-01,Quito,Pichincha,D,13
3,1,Beverages,0.00,0.00,2013-01-01,Quito,Pichincha,D,13
4,1,Books,0.00,0.00,2013-01-01,Quito,Pichincha,D,13
...,...,...,...,...,...,...,...,...,...
3008011,9,Poultry,0.00,0.00,2016-12-25,Quito,Pichincha,B,6
3008012,9,Prepared Foods,0.00,0.00,2016-12-25,Quito,Pichincha,B,6
3008013,9,Produce,0.00,0.00,2016-12-25,Quito,Pichincha,B,6
3008014,9,School And Office Supplies,0.00,0.00,2016-12-25,Quito,Pichincha,B,6


In [254]:
# Grouping the combined dataframe by city
train_by_city = train_x_stores.groupby(by="city").sales.agg(
    ["sum"]).reset_index().sort_values(by="sum", ascending=False)
train_by_city

,city,sum
18,Quito,"556,741,836.83"
8,Guayaquil,"122,967,260.48"
3,Cuenca,"49,168,596.03"
0,Ambato,"40,304,401.30"
21,Santo Domingo,"35,834,323.04"
13,Machala,"33,386,796.54"
2,Cayambe,"28,325,558.48"
10,Latacunga,"21,182,263.78"
4,Daule,"19,188,021.38"
12,Loja,"18,859,999.26"


In [255]:
# Finding the number of stores per city
stores_per_city = stores_data.groupby(
    by="city").store_nbr.agg(["count"]).reset_index()
stores_per_city

,city,count
0,Ambato,2
1,Babahoyo,1
2,Cayambe,1
3,Cuenca,3
4,Daule,1
5,El Carmen,1
6,Esmeraldas,1
7,Guaranda,1
8,Guayaquil,8
9,Ibarra,1


In [256]:
# Joining the dataframe on sales per store with the dataframe with number of stores per city
train_by_city_comp = pd.merge(train_by_city, stores_per_city)
train_by_city_comp.head(10)

,city,sum,count
0,Quito,"556,741,836.83",18
1,Guayaquil,"122,967,260.48",8
2,Cuenca,"49,168,596.03",3
3,Ambato,"40,304,401.30",2
4,Santo Domingo,"35,834,323.04",3
5,Machala,"33,386,796.54",2
6,Cayambe,"28,325,558.48",1
7,Latacunga,"21,182,263.78",2
8,Daule,"19,188,021.38",1
9,Loja,"18,859,999.26",1


In [257]:
# Calculating an average total sales per store per city
train_by_city_comp["mean_sales"] = train_by_city_comp["sum"] / \
    train_by_city_comp["count"]
train_by_city_comp.head(10)

,city,sum,count,mean_sales
0,Quito,"556,741,836.83",18,"30,930,102.05"
1,Guayaquil,"122,967,260.48",8,"15,370,907.56"
2,Cuenca,"49,168,596.03",3,"16,389,532.01"
3,Ambato,"40,304,401.30",2,"20,152,200.65"
4,Santo Domingo,"35,834,323.04",3,"11,944,774.35"
5,Machala,"33,386,796.54",2,"16,693,398.27"
6,Cayambe,"28,325,558.48",1,"28,325,558.48"
7,Latacunga,"21,182,263.78",2,"10,591,131.89"
8,Daule,"19,188,021.38",1,"19,188,021.38"
9,Loja,"18,859,999.26",1,"18,859,999.26"


In [258]:
# Calculating the average sales per city
train_by_city_avg = train_by_city_comp.sort_values(
    by="mean_sales", ascending=False)
train_by_city_avg.head(10)

,city,sum,count,mean_sales
0,Quito,"556,741,836.83",18,"30,930,102.05"
6,Cayambe,"28,325,558.48",1,"28,325,558.48"
3,Ambato,"40,304,401.30",2,"20,152,200.65"
8,Daule,"19,188,021.38",1,"19,188,021.38"
9,Loja,"18,859,999.26",1,"18,859,999.26"
10,Babahoyo,"17,746,872.70",1,"17,746,872.70"
5,Machala,"33,386,796.54",2,"16,693,398.27"
11,Esmeraldas,"16,391,692.21",1,"16,391,692.21"
2,Cuenca,"49,168,596.03",3,"16,389,532.01"
1,Guayaquil,"122,967,260.48",8,"15,370,907.56"


In [259]:
# Visualizing the total sales performance over the period by city
train_by_city_comp["count"] = train_by_city_comp["count"].apply(str)

fig = px.bar(train_by_city_comp, x="city", y="sum", title="Total Sales Performance by City",
             labels={"city": "City", "sum": "Total Sales", "count": "Stores in City"})
fig.show()

In [260]:
# Visualizing the average sales performance over the period by city
fig = px.bar(train_by_city_avg, x="city", y="mean_sales",
             title="Performance by City: Average Sales per Store",
             labels={"city": "City", "mean_sales": "Average Sales", "count": "Stores in City"})
fig.show()

Looking at the total sales per city, Quito (USD 556.74m), Guayaquil (USD 122.97m), Cuenca (USD 49.17m), Ambato (USD 40.30m), and Santo Domingo (USD 35.83m) are the best performing cities. This, however, may be misleading when you look at the average sales per city.

Using the average sales per city, Quito (USD 30.93m) still leads but this time it is followed by Cayambe (USD 28.33m), Ambato (USD 20.15m), Daule (19.19m) and Loja (USD 18.86m) to make the top 5. 

Quito's superb performance both overall and on average may be largely due to it's numerical advantage, but why can same not be said for Guayaquil which has 8 stores with an average of USD 15.37m per store? The same question can be asked of Cuenca, which placed 9th by average sales with USD 16.39m, Santa Domingo (14th), and Machala (7th).

With these in mind, it is recommended that - subject to further research and considerations - Corporation Favorita should consider opening more branches in larger cities which have higher average sales per city so as to maximize the numerical advantage as is done by Quito.

*What is the relationship between the number of stores and the total sales per city?*

In [261]:
# Visualizing the relationship between the number of stores per city and the total sales per city
train_by_city_comp["count"] = train_by_city_comp["count"].apply(int)

fig = px.scatter(train_by_city_comp, x="count", y="sum", trendline="ols",
                 title="Relationship between number of stores and total sales per city",
                 labels={"city": "City", "sum": "Total Sales", "count": "Stores in City"})
fig.show()

With an R2 value of 0.9519, the number of stores can be said to have a strong positive relationship with the total sales per city. But looking at the plot above, we note that there is an obvious outlier, which is likely to have had an impact on the trendline.

To get a better view of the relationship, we will exclude the city with 18 stores from the dataframe and then re-plot to see a "truer relationship" 

In [262]:
# Visualizing the relationship between the number of stores per city and the total sales per city without the "outlier"
fig = px.scatter(train_by_city_comp[1:], x="count", y="sum", trendline="ols",
                 title="Relationship between number of stores and total sales per city",
                 labels={"city": "City", "sum": "Total Sales", "count": "Stores in City"})
fig.show()

Here also we see that the city with 8 stores is an outlier on the plot, so despite having an R2 of 0.9230 (strong positive relationship between the variables) we will also exclude the city with 8 stores from the plot to assess the relationship.

In [263]:
# Visualizing the relationship between the number of stores per city and the total sales per city without the two "outliers"
fig = px.scatter(train_by_city_comp[2:], x="count", y="sum", trendline="ols",
                 title="Relationship between number of stores and total sales per city",
                 labels={"city": "City", "sum": "Total Sales", "count": "Stores in City"})
fig.show()

With the exclusion of the two outliers, the "final" plot still shows a positive correlation between the number of stores and the  total sales per city. It has an R-squared value of 0.6352, indicating a fair positive relationship between the variables.

***Analysis of sales by state***

*How does the sales performance by state look like?*

In [264]:
# Grouping the combined dataframe by state
train_by_state = train_x_stores.groupby(by="state").sales.agg(["sum"]
                                                              ).sort_values(by="sum", ascending=False).reset_index()
train_by_state

,state,sum
0,Pichincha,"585,067,395.31"
1,Guayas,"165,154,392.38"
2,Azuay,"49,168,596.03"
3,Tungurahua,"40,304,401.30"
4,Santo Domingo de los Tsachilas,"35,834,323.04"
5,El Oro,"33,386,796.54"
6,Los Rios,"31,925,373.99"
7,Manabi,"24,969,820.13"
8,Cotopaxi,"21,182,263.78"
9,Loja,"18,859,999.26"


In [265]:
# Finding the number of stores per sta
stores_per_state = stores_data.groupby(
    by="state").store_nbr.agg(["count"]).reset_index()
stores_per_state

,state,count
0,Azuay,3
1,Bolivar,1
2,Chimborazo,1
3,Cotopaxi,2
4,El Oro,2
5,Esmeraldas,1
6,Guayas,11
7,Imbabura,1
8,Loja,1
9,Los Rios,2


In [266]:
# Joining the dataframe on sales per store with the dataframe with number of stores per state
train_by_state_comp = pd.merge(train_by_state, stores_per_state)
train_by_state_comp

,state,sum,count
0,Pichincha,"585,067,395.31",19
1,Guayas,"165,154,392.38",11
2,Azuay,"49,168,596.03",3
3,Tungurahua,"40,304,401.30",2
4,Santo Domingo de los Tsachilas,"35,834,323.04",3
5,El Oro,"33,386,796.54",2
6,Los Rios,"31,925,373.99",2
7,Manabi,"24,969,820.13",3
8,Cotopaxi,"21,182,263.78",2
9,Loja,"18,859,999.26",1


In [267]:
# Calculating an average total sales per store per city
train_by_state_comp["avg_per_store"] = train_by_state_comp["sum"] / \
    train_by_state_comp["count"]
train_by_state_comp

,state,sum,count,avg_per_store
0,Pichincha,"585,067,395.31",19,"30,793,020.81"
1,Guayas,"165,154,392.38",11,"15,014,035.67"
2,Azuay,"49,168,596.03",3,"16,389,532.01"
3,Tungurahua,"40,304,401.30",2,"20,152,200.65"
4,Santo Domingo de los Tsachilas,"35,834,323.04",3,"11,944,774.35"
5,El Oro,"33,386,796.54",2,"16,693,398.27"
6,Los Rios,"31,925,373.99",2,"15,962,687.00"
7,Manabi,"24,969,820.13",3,"8,323,273.38"
8,Cotopaxi,"21,182,263.78",2,"10,591,131.89"
9,Loja,"18,859,999.26",1,"18,859,999.26"


In [268]:
# Calculating the average sales per state
train_by_state_avg = train_by_state_comp.sort_values(
    by="avg_per_store", ascending=False)
train_by_state_avg

,state,sum,count,avg_per_store
0,Pichincha,"585,067,395.31",19,"30,793,020.81"
3,Tungurahua,"40,304,401.30",2,"20,152,200.65"
9,Loja,"18,859,999.26",1,"18,859,999.26"
5,El Oro,"33,386,796.54",2,"16,693,398.27"
10,Esmeraldas,"16,391,692.21",1,"16,391,692.21"
2,Azuay,"49,168,596.03",3,"16,389,532.01"
6,Los Rios,"31,925,373.99",2,"15,962,687.00"
1,Guayas,"165,154,392.38",11,"15,014,035.67"
11,Bolivar,"13,049,509.84",1,"13,049,509.84"
4,Santo Domingo de los Tsachilas,"35,834,323.04",3,"11,944,774.35"


In [269]:
# Visualizing the total sales performance over the period by state
train_by_state_comp["count"] = train_by_state_comp["count"].apply(str)

fig = px.bar(train_by_state_comp, x="state", y="sum", title="Total Sales Performance by State",
             labels={"state": "State", "sum": "Total Sales"})
fig.show()

In [270]:
# Visualizing the average sales performance over the period by state
fig = px.bar(train_by_state_avg, x="state", y="avg_per_store",
             title="Performance by State: Average Sales per Store",
             labels={"state": "State", "avg_per_store": "Average Sales"})
fig.show()

*What is the relationship between the number of stores and the total sales per state?*

In [271]:
# Visualizing the relationship between the number of stores and the total sales per state
train_by_state_comp["count"] = train_by_state_comp["count"].apply(int)

fig = px.scatter(train_by_state_comp, x="count", y="sum", trendline="ols",
                 title="Relationship between number of stores and total sales per state",
                 labels={"state": "State", "sum": "Total Sales", "count": "Stores in State"})
fig.show()

With an R2 value of 0.9267, the number of stores can be said to have a strong positive relationship with the total sales per state. But the plot above, shows an outlier which is likely to have had an impact on the trendline.

To get a better view of the relationship, we will exclude the state with 19 stores from the dataframe and then re-plot to see a "truer relationship".

In [272]:
train_by_state_comp

,state,sum,count,avg_per_store
0,Pichincha,"585,067,395.31",19,"30,793,020.81"
1,Guayas,"165,154,392.38",11,"15,014,035.67"
2,Azuay,"49,168,596.03",3,"16,389,532.01"
3,Tungurahua,"40,304,401.30",2,"20,152,200.65"
4,Santo Domingo de los Tsachilas,"35,834,323.04",3,"11,944,774.35"
5,El Oro,"33,386,796.54",2,"16,693,398.27"
6,Los Rios,"31,925,373.99",2,"15,962,687.00"
7,Manabi,"24,969,820.13",3,"8,323,273.38"
8,Cotopaxi,"21,182,263.78",2,"10,591,131.89"
9,Loja,"18,859,999.26",1,"18,859,999.26"


In [273]:
# Visualizing the relationship between the number of stores and the total sales per state without the "outlier"
fig = px.scatter(train_by_state_comp[1:], x="count", y="sum", trendline="ols",
                 title="Relationship between number of stores and total sales per state",
                 labels={"state": "State", "sum": "Total Sales", "count": "Stores in State"})
fig.show()

From the plot, we may conclude that there is no relationship between the number of stores per state and the total sales from that state.

**What defines a store type?**

In [274]:
store_performance = train_data.groupby(
    by="store_nbr").sales.agg(["sum"]).reset_index()
store_performance.head()

,store_nbr,sum
0,1,"14,145,012.87"
1,2,"21,557,388.64"
2,3,"50,481,910.19"
3,4,"18,909,700.38"
4,5,"15,592,404.57"


In [275]:
stores_data

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4
5,6,Quito,Pichincha,D,13
6,7,Quito,Pichincha,D,8
7,8,Quito,Pichincha,D,8
8,9,Quito,Pichincha,B,6
9,10,Quito,Pichincha,C,15


In [276]:
store_details_with_performance = pd.merge(stores_data, store_performance)
store_details_with_performance.rename(
    columns={"sum": "total_sales"}, inplace=True)
store_details_with_performance

,store_nbr,city,state,type,cluster,total_sales
0,1,Quito,Pichincha,D,13,"14,145,012.87"
1,2,Quito,Pichincha,D,13,"21,557,388.64"
2,3,Quito,Pichincha,D,8,"50,481,910.19"
3,4,Quito,Pichincha,D,9,"18,909,700.38"
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4,"15,592,404.57"
5,6,Quito,Pichincha,D,13,"25,183,680.00"
6,7,Quito,Pichincha,D,8,"26,952,049.58"
7,8,Quito,Pichincha,D,8,"30,494,286.93"
8,9,Quito,Pichincha,B,6,"26,409,441.06"
9,10,Quito,Pichincha,C,15,"9,613,905.74"


In [277]:
store_perf_by_type = store_details_with_performance.groupby(
    "type").total_sales.agg(["count", "sum", "mean"]).reset_index()
store_perf_by_type

,type,count,sum,mean
0,A,9,"353,043,841.80","39,227,093.53"
1,B,8,"145,260,635.13","18,157,579.39"
2,C,15,"164,434,742.56","10,962,316.17"
3,D,18,"351,083,296.86","19,504,627.60"
4,E,4,"59,822,435.84","14,955,608.96"


From the table above, the nature of the relationship between the type of store and sales is unclear as type A which has 9 stores has more sales than Type D which had the most stores.

## Feature Engineering

**Merging all the dataframes**

In [278]:
# Taking a look at the training data before beginning the mergers
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3008016 entries, 0 to 3008015
Data columns (total 5 columns):
 #   Column       Dtype  
---  ------       -----  
 0   store_nbr    int64  
 1   family       object 
 2   sales        float64
 3   onpromotion  float64
 4   sales_date   object 
dtypes: float64(2), int64(1), object(2)
memory usage: 114.7+ MB


In [279]:
# Merging the train data with the other dataframes
merged_data = pd.merge(train_data, stores_data)  # with the stores data
merged_data = pd.merge(merged_data, oil_data,
                       left_on="sales_date", right_on="date")  # with the oil data
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3008016 entries, 0 to 3008015
Data columns (total 11 columns):
 #   Column       Dtype  
---  ------       -----  
 0   store_nbr    int64  
 1   family       object 
 2   sales        float64
 3   onpromotion  float64
 4   sales_date   object 
 5   city         object 
 6   state        object 
 7   type         object 
 8   cluster      int64  
 9   date         object 
 10  dcoilwtico   float64
dtypes: float64(3), int64(2), object(6)
memory usage: 275.4+ MB


In [280]:
# Ensuring same date formats with holiday data before merging
merged_data["sales_date"] = pd.to_datetime(merged_data["sales_date"]).dt.date
holidays_events["sales_date"] = pd.to_datetime(holidays_events["date"]).dt.date
holidays_events.drop(columns= ["date"], inplace= True)

In [281]:
holidays_events["locale_name"].value_counts()

Ecuador                           174
Quito                              13
Riobamba                           12
Guaranda                           12
Latacunga                          12
Ambato                             12
Guayaquil                          11
Cuenca                              7
Ibarra                              7
Salinas                             6
Loja                                6
Santa Elena                         6
Santo Domingo de los Tsachilas      6
Quevedo                             6
Manta                               6
Esmeraldas                          6
Cotopaxi                            6
El Carmen                           6
Santo Domingo                       6
Machala                             6
Imbabura                            6
Puyo                                6
Libertad                            6
Cayambe                             6
Name: locale_name, dtype: int64

In [282]:
# Merging with the holidays data
merged_data = pd.merge(merged_data, holidays_events, on="sales_date")
merged_data

,store_nbr,family,sales,onpromotion,sales_date,city,state,type_x,cluster,date,dcoilwtico,type_y,locale,locale_name,description,transferred
0,1,Automotive,0.00,0.00,2013-01-01,Quito,Pichincha,D,13,2013-01-01,93.14,Holiday,National,Ecuador,Primer dia del ano,False
1,1,Baby Care,0.00,0.00,2013-01-01,Quito,Pichincha,D,13,2013-01-01,93.14,Holiday,National,Ecuador,Primer dia del ano,False
2,1,Beauty,0.00,0.00,2013-01-01,Quito,Pichincha,D,13,2013-01-01,93.14,Holiday,National,Ecuador,Primer dia del ano,False
3,1,Beverages,0.00,0.00,2013-01-01,Quito,Pichincha,D,13,2013-01-01,93.14,Holiday,National,Ecuador,Primer dia del ano,False
4,1,Books,0.00,0.00,2013-01-01,Quito,Pichincha,D,13,2013-01-01,93.14,Holiday,National,Ecuador,Primer dia del ano,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3061471,9,Poultry,0.00,0.00,2016-12-25,Quito,Pichincha,B,6,2016-12-25,52.01,Holiday,National,Ecuador,Navidad,False
3061472,9,Prepared Foods,0.00,0.00,2016-12-25,Quito,Pichincha,B,6,2016-12-25,52.01,Holiday,National,Ecuador,Navidad,False
3061473,9,Produce,0.00,0.00,2016-12-25,Quito,Pichincha,B,6,2016-12-25,52.01,Holiday,National,Ecuador,Navidad,False
3061474,9,School And Office Supplies,0.00,0.00,2016-12-25,Quito,Pichincha,B,6,2016-12-25,52.01,Holiday,National,Ecuador,Navidad,False


In [283]:
# Looking at the nulls in the current version of the merged dataframe
merged_data.isnull().sum()

store_nbr            0
family               0
sales                0
onpromotion          0
sales_date           0
city                 0
state                0
type_x               0
cluster              0
date                 0
dcoilwtico           0
type_y         2551824
locale         2551824
locale_name    2551824
description    2551824
transferred    2551824
dtype: int64

We notice a large number of nulls, for the days that were not holidays so we replace with some values.

In [284]:
merged_data["type_y"].unique()

array(['Holiday', nan, 'Work Day', 'Additional', 'Event', 'Transfer',
       'Bridge'], dtype=object)

In [285]:
# Looking at the summary info of dates with type "Work Day"
work_day = merged_data[merged_data["type_y"] == "Work Day"]
work_day

,store_nbr,family,sales,onpromotion,sales_date,city,state,type_x,cluster,date,dcoilwtico,type_y,locale,locale_name,description,transferred
7128,1,Automotive,5.00,0.00,2013-01-05,Quito,Pichincha,D,13,2013-01-05,93.12,Work Day,National,Ecuador,Recupero puente Navidad,False
7129,1,Baby Care,0.00,0.00,2013-01-05,Quito,Pichincha,D,13,2013-01-05,93.12,Work Day,National,Ecuador,Recupero puente Navidad,False
7130,1,Beauty,3.00,0.00,2013-01-05,Quito,Pichincha,D,13,2013-01-05,93.12,Work Day,National,Ecuador,Recupero puente Navidad,False
7131,1,Beverages,"1,160.00",0.00,2013-01-05,Quito,Pichincha,D,13,2013-01-05,93.12,Work Day,National,Ecuador,Recupero puente Navidad,False
7132,1,Books,0.00,0.00,2013-01-05,Quito,Pichincha,D,13,2013-01-05,93.12,Work Day,National,Ecuador,Recupero puente Navidad,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555379,9,Poultry,529.68,1.00,2016-11-12,Quito,Pichincha,B,6,2016-11-12,43.39,Work Day,National,Ecuador,Recupero Puente Dia de Difuntos,False
2555381,9,Prepared Foods,158.77,2.00,2016-11-12,Quito,Pichincha,B,6,2016-11-12,43.39,Work Day,National,Ecuador,Recupero Puente Dia de Difuntos,False
2555383,9,Produce,"1,618.00",2.00,2016-11-12,Quito,Pichincha,B,6,2016-11-12,43.39,Work Day,National,Ecuador,Recupero Puente Dia de Difuntos,False
2555385,9,School And Office Supplies,3.00,0.00,2016-11-12,Quito,Pichincha,B,6,2016-11-12,43.39,Work Day,National,Ecuador,Recupero Puente Dia de Difuntos,False


In [286]:
work_day.nunique()

store_nbr        54
family           33
sales          2524
onpromotion      91
sales_date        5
city             22
state            16
type_x            5
cluster          17
date              5
dcoilwtico        5
type_y            1
locale            1
locale_name       1
description       5
transferred       1
dtype: int64

When we look at the summary data, we note that the locale value is national and transferred is false for all work days. This will be replicated across the merged dataframe to fill in the nulls for the missing dates which were not holidays.

In [287]:
# Filling the nulls in the holiday data
merged_data["type_y"] = merged_data["type_y"].fillna("Work Day")
merged_data["locale"] = merged_data["locale"].fillna("National")
merged_data["transferred"] = merged_data["transferred"].fillna(False)
merged_data.isnull().sum()

store_nbr            0
family               0
sales                0
onpromotion          0
sales_date           0
city                 0
state                0
type_x               0
cluster              0
date                 0
dcoilwtico           0
type_y               0
locale               0
locale_name    2551824
description    2551824
transferred          0
dtype: int64

In [288]:
# Dropping columns that will not be used
merged_data.drop(columns=["date", "locale_name", "description"], inplace=True)

# Renaming confusing column names
merged_data.rename(columns= {"type_x": "store_type",
                            "type_y": "holiday_type", 
                            "dcoilwtico": "oil_price"}, 
                   inplace= True)
merged_data

,store_nbr,family,sales,onpromotion,sales_date,city,state,store_type,cluster,oil_price,holiday_type,locale,transferred
0,1,Automotive,0.00,0.00,2013-01-01,Quito,Pichincha,D,13,93.14,Holiday,National,False
1,1,Baby Care,0.00,0.00,2013-01-01,Quito,Pichincha,D,13,93.14,Holiday,National,False
2,1,Beauty,0.00,0.00,2013-01-01,Quito,Pichincha,D,13,93.14,Holiday,National,False
3,1,Beverages,0.00,0.00,2013-01-01,Quito,Pichincha,D,13,93.14,Holiday,National,False
4,1,Books,0.00,0.00,2013-01-01,Quito,Pichincha,D,13,93.14,Holiday,National,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3061471,9,Poultry,0.00,0.00,2016-12-25,Quito,Pichincha,B,6,52.01,Holiday,National,False
3061472,9,Prepared Foods,0.00,0.00,2016-12-25,Quito,Pichincha,B,6,52.01,Holiday,National,False
3061473,9,Produce,0.00,0.00,2016-12-25,Quito,Pichincha,B,6,52.01,Holiday,National,False
3061474,9,School And Office Supplies,0.00,0.00,2016-12-25,Quito,Pichincha,B,6,52.01,Holiday,National,False


In [289]:
merged_data.nunique()

store_nbr           54
family              33
sales           379610
onpromotion        362
sales_date        1688
city                22
state               16
store_type           5
cluster             17
oil_price         1032
holiday_type         6
locale               3
transferred          2
dtype: int64

In [290]:
# Looking at the correlation between the variables in the merged dataframe
correlation = pd.DataFrame(merged_data.corr())
fig = px.imshow(correlation, text_auto=True, aspect="auto")
fig.show()

In [291]:
# Removing the restriction on columns to display
pd.set_option("display.max_columns", None)

In [292]:
merged_data.info(show_counts= True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3061476 entries, 0 to 3061475
Data columns (total 13 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   store_nbr     3061476 non-null  int64  
 1   family        3061476 non-null  object 
 2   sales         3061476 non-null  float64
 3   onpromotion   3061476 non-null  float64
 4   sales_date    3061476 non-null  object 
 5   city          3061476 non-null  object 
 6   state         3061476 non-null  object 
 7   store_type    3061476 non-null  object 
 8   cluster       3061476 non-null  int64  
 9   oil_price     3061476 non-null  float64
 10  holiday_type  3061476 non-null  object 
 11  locale        3061476 non-null  object 
 12  transferred   3061476 non-null  bool   
dtypes: bool(1), float64(3), int64(2), object(7)
memory usage: 306.6+ MB


In [293]:
# Defining a function to get date features from dataframe
def getDateFeatures(df, date):
    df["date"] = pd.to_datetime(df[date])
    df["day_of_week"] = df["date"].dt.dayofweek.astype(int)
    df["day_of_month"] = df["date"].dt.day.astype(int)
    df["day_of_year"] = df["date"].dt.dayofyear.astype(int)
    df["is_weekend"] = np.where(df["day_of_week"] > 4, 1, 0).astype(int)

    df["week_of_year"] = df["date"].dt.isocalendar().week.astype(int)

    df["month"] = df["date"].dt.month.astype(int)

    df["is_year_start"] = df["date"].dt.is_year_end.astype(int)
    df["is_year_end"] = df["date"].dt.is_year_end.astype(int)
    df["year"] = df["date"].dt.year.astype(int)
    df = df.drop(columns="date")

    return df

In [294]:
# Getting the date features from the Train Dataframe
final_train = getDateFeatures(merged_data, "sales_date")
final_train = final_train.sort_values(by=["sales_date", "store_nbr"], 
                                      ignore_index= True)
final_train

,store_nbr,family,sales,onpromotion,sales_date,city,state,store_type,cluster,oil_price,holiday_type,locale,transferred,day_of_week,day_of_month,day_of_year,is_weekend,week_of_year,month,is_year_start,is_year_end,year
0,1,Automotive,0.00,0.00,2013-01-01,Quito,Pichincha,D,13,93.14,Holiday,National,False,1,1,1,0,1,1,0,0,2013
1,1,Baby Care,0.00,0.00,2013-01-01,Quito,Pichincha,D,13,93.14,Holiday,National,False,1,1,1,0,1,1,0,0,2013
2,1,Beauty,0.00,0.00,2013-01-01,Quito,Pichincha,D,13,93.14,Holiday,National,False,1,1,1,0,1,1,0,0,2013
3,1,Beverages,0.00,0.00,2013-01-01,Quito,Pichincha,D,13,93.14,Holiday,National,False,1,1,1,0,1,1,0,0,2013
4,1,Books,0.00,0.00,2013-01-01,Quito,Pichincha,D,13,93.14,Holiday,National,False,1,1,1,0,1,1,0,0,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3061471,54,Poultry,59.62,0.00,2017-08-15,El Carmen,Manabi,C,3,47.57,Holiday,Local,False,1,15,227,0,33,8,0,0,2017
3061472,54,Prepared Foods,94.00,0.00,2017-08-15,El Carmen,Manabi,C,3,47.57,Holiday,Local,False,1,15,227,0,33,8,0,0,2017
3061473,54,Produce,915.37,76.00,2017-08-15,El Carmen,Manabi,C,3,47.57,Holiday,Local,False,1,15,227,0,33,8,0,0,2017
3061474,54,School And Office Supplies,0.00,0.00,2017-08-15,El Carmen,Manabi,C,3,47.57,Holiday,Local,False,1,15,227,0,33,8,0,0,2017


In [295]:
# Encoding the categorical columns
categoricals = merged_data.select_dtypes(include=["object"]).columns.to_list()
categoricals.remove("sales_date")
categoricals

['family', 'city', 'state', 'store_type', 'holiday_type', 'locale']

In [296]:
# Encoding the categorical variables
oh_encoder = OneHotEncoder(drop = "first", sparse = False)
oh_encoder.fit(merged_data[categoricals])

encoded_categoricals = oh_encoder.transform(merged_data[categoricals])
encoded_categoricals = pd.DataFrame(encoded_categoricals, columns = oh_encoder.get_feature_names_out().tolist())
encoded_categoricals

,family_Baby Care,family_Beauty,family_Beverages,family_Books,family_Bread/Bakery,family_Celebration,family_Cleaning,family_Dairy,family_Deli,family_Eggs,family_Frozen Foods,family_Grocery I,family_Grocery Ii,family_Hardware,family_Home And Kitchen I,family_Home And Kitchen Ii,family_Home Appliances,family_Home Care,family_Ladieswear,family_Lawn And Garden,family_Lingerie,"family_Liquor,Wine,Beer",family_Magazines,family_Meats,family_Personal Care,family_Pet Supplies,family_Players And Electronics,family_Poultry,family_Prepared Foods,family_Produce,family_School And Office Supplies,family_Seafood,city_Babahoyo,city_Cayambe,city_Cuenca,city_Daule,city_El Carmen,city_Esmeraldas,city_Guaranda,city_Guayaquil,city_Ibarra,city_Latacunga,city_Libertad,city_Loja,city_Machala,city_Manta,city_Playas,city_Puyo,city_Quevedo,city_Quito,city_Riobamba,city_Salinas,city_Santo Domingo,state_Bolivar,state_Chimborazo,state_Cotopaxi,state_El Oro,state_Esmeraldas,state_Guayas,state_Imbabura,state_Loja,state_Los Rios,state_Manabi,state_Pastaza,state_Pichincha,state_Santa Elena,state_Santo Domingo de los Tsachilas,state_Tungurahua,store_type_B,store_type_C,store_type_D,store_type_E,holiday_type_Bridge,holiday_type_Event,holiday_type_Holiday,holiday_type_Transfer,holiday_type_Work Day,locale_National,locale_Regional
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00
1,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00
2,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00
3,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00
4,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3061471,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.0

In [297]:
# Adding the encoded categoricals to the DataFrame and dropping the original columns
final_train = final_train.join(encoded_categoricals)
final_train.drop(columns= categoricals, inplace= True)
final_train.rename(columns=lambda x: re.sub('[^A-Za-z0-9_]+', '', x), inplace= True)
final_train

,store_nbr,sales,onpromotion,sales_date,cluster,oil_price,transferred,day_of_week,day_of_month,day_of_year,is_weekend,week_of_year,month,is_year_start,is_year_end,year,family_BabyCare,family_Beauty,family_Beverages,family_Books,family_BreadBakery,family_Celebration,family_Cleaning,family_Dairy,family_Deli,family_Eggs,family_FrozenFoods,family_GroceryI,family_GroceryIi,family_Hardware,family_HomeAndKitchenI,family_HomeAndKitchenIi,family_HomeAppliances,family_HomeCare,family_Ladieswear,family_LawnAndGarden,family_Lingerie,family_LiquorWineBeer,family_Magazines,family_Meats,family_PersonalCare,family_PetSupplies,family_PlayersAndElectronics,family_Poultry,family_PreparedFoods,family_Produce,family_SchoolAndOfficeSupplies,family_Seafood,city_Babahoyo,city_Cayambe,city_Cuenca,city_Daule,city_ElCarmen,city_Esmeraldas,city_Guaranda,city_Guayaquil,city_Ibarra,city_Latacunga,city_Libertad,city_Loja,city_Machala,city_Manta,city_Playas,city_Puyo,city_Quevedo,city_Quito,city_Riobamba,city_Salinas,city_SantoDomingo,state_Bolivar,state_Chimborazo,state_Cotopaxi,state_ElOro,state_Esmeraldas,state_Guayas,state_Imbabura,state_Loja,state_LosRios,state_Manabi,state_Pastaza,state_Pichincha,state_SantaElena,state_SantoDomingodelosTsachilas,state_Tungurahua,store_type_B,store_type_C,store_type_D,store_type_E,holiday_type_Bridge,holiday_type_Event,holiday_type_Holiday,holiday_type_Transfer,holiday_type_WorkDay,locale_National,locale_Regional
0,1,0.00,0.00,2013-01-01,13,93.14,False,1,1,1,0,1,1,0,0,2013,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00
1,1,0.00,0.00,2013-01-01,13,93.14,False,1,1,1,0,1,1,0,0,2013,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00
2,1,0.00,0.00,2013-01-01,13,93.14,False,1,1,1,0,1,1,0,0,2013,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00
3,1,0.00,0.00,2013-01-01,13,93.14,False,1,1,1,0,1,1,0,0,2013,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00
4,1,0.00,0.00,2013-01-01,13,93.14,False,1,1,1,0,1,1,0,0,2013,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [298]:
# Looking at the correlation between the variables in the final dataframe
correlation = pd.DataFrame(final_train.corr())
fig = px.imshow(correlation, text_auto= True, aspect= "auto")
fig.show()

## Modelling

In [299]:
# Splitting the data into train and validation
train = final_train.loc[final_train["year"].isin([2013, 2014, 2015, 2016])]
validation = final_train.loc[final_train["year"].isin([2017])]

# Dropping sales date and transferred columns
train.drop(columns = ["sales_date", "transferred"], inplace = True)
validation.drop(columns = ["sales_date", "transferred"], inplace = True)

In [301]:
# Training data
X = train.drop(columns = "sales")
y = train["sales"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.25, random_state= 24)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(1987375, 92) (1987375,)
(662459, 92) (662459,)


In [302]:
# Defining which columns to scale
col_to_scale = ["onpromotion"]

# Fitting to the training data
mm_scaler = MinMaxScaler()
mm_scaler.fit(X_train[col_to_scale])

# Transforming the features
X_train[col_to_scale] = mm_scaler.transform(X_train[col_to_scale])
X_test[col_to_scale] = mm_scaler.transform(X_test[col_to_scale])

In [303]:
# Defining the models to use
models = {"Decision Tree": DecisionTreeRegressor(random_state = 24),
          #"Random Forest": RandomForestRegressor(random_state = 24),
          "XGBoost": XGBRegressor(random_state = 24)
         }

In [313]:
# Defining a helper function to fit models to data and score them
def fit_and_score(models, X_train = X_train, X_test = X_test, y_train = y_train, y_test = y_test):
    
    # A dictionary to keep the scores
    model_scores = {}
    model_mses = {}
    model_rmses = {}
    model_rmsles = {}
    
    # Looping through the models to fit and score each
    for name, model in models.items():
        # model training and evaluation start time
        training_start = pd.datetime.now()
        
        # fitting to the training data
        model.fit(X_train, y_train)
        
        # getting the feature importances
        model_feature_importances = pd.DataFrame(model.feature_importances_, columns=["importance_level"]).reset_index()
        model_feature_importances["Feature"] = list(X_train.columns)
        model_feature_importances.drop(columns=["index"], inplace= True)
        model_feature_importances.sort_values(by="importance_level", ascending= False, inplace = True)
        
        # visualizing the feature importances
        fig = px.bar(model_feature_importances.head(20),
                     x= "Feature", y= "importance_level",
                     title = f"{name} Feature Importances")
        fig.show()
        
        # making predictions
        y_pred = model.predict(X_test)
        
        # calculating model evaluation metrics
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        
        # checking if there are any negative predictions before calculation RMSLE
        negative_preds = len(y_pred[y_pred < 0])
        if negative_preds <= 0.00:
            rmsle = np.sqrt(mean_squared_log_error(y_test, y_pred))
        else: rmsle = np.nan

        # evaluating the model and putting the evaluation metrics together
        model_scores[name] = model.score(X_test, y_test)
        model_mses[name] = mse
        model_rmses[name] = rmse
        model_rmsles[name] = negative_predictions

        # model training and evaluation end time
        training_end = pd.datetime.now()
        
        # total time and evaluating time
        train_duration = training_end - training_start
        print(name, " training duration: ", train_duration)
        
    summary = pd.DataFrame([model_scores, model_mses, model_rmses, model_rmsles]).T
    summary.columns = ["R2", "MSE", "RMSE", "RMSLE"]
    #return model_scores, model_mses, model_rmses
    return summary

In [314]:
model_performances = fit_and_score(models, X_train, X_test, y_train, y_test)
model_performances

NameError: name 'prediction' is not defined

In [ ]:
# Predicting and Evaluating the Decision Tree model
rmsle = np.sqrt(mean_squared_log_error(y_test, prediction))
print("RMSLE: ", rmsle)

### Model 2: Random Forest Model

In [ ]:
# Random Forest
rf_model = RandomForestRegressor(random_state= 24)
rf_model = rf_reg.fit(X_train, y_train)

In [ ]:
# Predicting and Evaluating the Random Forest model
y_pred = rf_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
rmsle = np.sqrt(mean_squared_log_error(y_test, y_pred))
print("Mean-Squared Error:", mse)
print("Root Mean-Squared Error:", rmse)
print("Root Mean-Squared Log Error:", rmsle)

## Exporting

In [162]:
# Exporting the requirements
requirements = "\n".join(f"{m.__name__}=={m.__version__}" for m in globals().values() if getattr(m, "__version__", None))

with open("requirements.txt", "w") as f:
    f.write(requirements)

In [166]:
# Creating a dictionary of objects to export
exports = {
    "encoder": oh_encoder,
    "scaler": mm_scaler,
    "model": dt_model
}

In [167]:
# Exporting the dictionary with Pickle
with open("ML_toolkit", "wb") as file:
    pickle.dump(exports, file)